# Data Cleaning

This code base was used to clean the sierra_db in order for the RAG to be more efficent. To be able to run it 
1. pip install -r requirments.txt
2. Create a .env file with "OPEN_AI_KEY" and you API key as value
3. Create a folder "data" in which you will put the raw dataset to clean

In [14]:
import pandas as pd

# Path to the CSV file
csv_file_path = 'data/sierra_db.csv'

# Read the CSV file with specified dtypes
data = pd.read_csv(csv_file_path, dtype={'column_name_2': str, 'column_name_3': str, 'column_name_4': str, 'column_name_6': str}, low_memory=False)

In [15]:
results = data[['UniqueUserReference', 'QuestionDate', 'UserQuestion']]
print("Total data:", len(results))

Total data: 94654


### 1, 2 Remove Duplicates and empty columns

In [16]:
# Call the OpenAI API and prompt with UserQuestion and store the ModelAnswer
# Display all duplicates
duplicates = results[results.duplicated()]
print(f"Number of duplicate rows: {len(duplicates)}")

# Keep only one instance of each duplicate
results = results.drop_duplicates()
print(f"Number of rows after removing duplicates: {len(results)}")

# Remove empty columns
results = results.dropna(axis=1, how='all')
print(f"Number of columns after removing empty columns: {len(results)}")

Number of duplicate rows: 14
Number of rows after removing duplicates: 94640
Number of columns after removing empty columns: 94640


### 3. Filter Very Short Questions

In [17]:
results = results[results['UserQuestion'].str.len() > 10]
print(f"Number of rows after filtering very short questions: {len(results)}")

Number of rows after filtering very short questions: 87925


### 4. Remove Spams

In [18]:
import re

def clean_text(text):
    # Remove excessive use of special characters
    text = re.sub(r'[@#$%&]+', '', text)
    # Remove repeated characters
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    return text

# Apply the clean_text function and filter out rows where the text was modified
results['CleanedUserQuestion'] = results['UserQuestion'].apply(clean_text)
results = results[results['UserQuestion'] == results['CleanedUserQuestion']]

# Drop the temporary 'CleanedUserQuestion' column
results = results.drop(columns=['CleanedUserQuestion'])

print(f"Number of rows after cleaning text: {len(results)}")

Number of rows after cleaning text: 86394


### 5. Remove Links and Emails

In [20]:
results = results[~results['UserQuestion'].str.contains(r'http://|https://|[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', na=False)]
print(f"Number of rows after removing links and emails: {len(results)}")

Number of rows after removing links and emails: 86380


### 6. Remove non English Words

In [21]:
from langdetect import detect

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

results = results[results['UserQuestion'].apply(is_english)]
print(f"Number of rows after removing non-English questions: {len(results)}")

Number of rows after removing non-English questions: 80250


## Cleaned Dataset is saved to sierra_db_cleaned,csv

In [ ]:
results.to_csv('sierra_db_cleaned.csv', index=False)

# Answer Generation

In [28]:
import openai
from tqdm import tqdm
import os

openai.api_key = os.getenv("OPEN_AI_KEY")

def get_model_answer(question):
    completion = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": question}
        ]
    )
    return completion.choices[0].message.content

### 1. Prompt each quesiton dated before 2024-10-15 16:22:50 trough gpt-4o-mini-2024-07-18

In [ ]:
sierra_db_4o_mini = pd.read_csv('sierra_db_4o_mini.csv')

for index, row in tqdm(sierra_db_4o_mini.iterrows(), total=sierra_db_4o_mini.shape[0]):
    if pd.isna(row['ModelAnswer']):
        question_date = pd.to_datetime(row['QuestionDate'])
        cutoff_date = pd.to_datetime('2024-10-15 16:22:50')
        
        if question_date > cutoff_date:
            model_answer = data.loc[data['UniqueUserReference'] == row['UniqueUserReference'], 'ModelAnswer'].values[0]
        else:
            question = row['UserQuestion']
            try:
                model_answer = get_model_answer(question)
                # Save the updated dataframe to the CSV file after each successful API call
                sierra_db_4o_mini.to_csv('sierra_db_4o_mini.csv', index=False)
            except Exception as e:
                print(f"Error processing row {index}: {e}")
                continue
        
        sierra_db_4o_mini.at[index, 'ModelAnswer'] = model_answer
